### Let's Do something with pagecounts

In [3]:
val pageCounts = sc.cassandraTable[(String,Int)]("databricks","pagecounts").select("project","requests")

In [4]:
pageCounts.first

(String, Int) = (lb.b,1)

### Let's sum requests by project

In [28]:
val sum_by_project = pageCounts.reduceByKey(_+_)

### Notice how long it takes to get the first row

In [29]:
sum_by_project.first

(String, Int) = (frr,462)

In [7]:
%%cql create table if not exists databricks.sum_by_project(project text primary key, sum_of_hits int)

In [8]:
sum_by_project.saveToCassandra("databricks", "sum_by_project")

In [9]:
%%cql select * from databricks.sum_by_project limit 10

project,sum_of_hits
lb.b,7
nah.mw,80
jv.d,226
fj.mw,83
av.mw,100
myv,336
kab.mw,37
ckb.mw,217
ru,1502263
is.d,421


### Note how we can use keyBy and spanByKey to remove the shuffle!
    - main row part is just one column pagecounts Int.  
    - keyBy is one column project.  Need to tack that onto the end of the select list

In [30]:
val pageCounts = sc.cassandraTable[Int]("databricks","pagecounts")
    .select("requests","project")
    .keyBy[String]("project")

In [31]:
pageCounts.first

(String, Int) = (lb.b,1)

In [32]:
pageCounts.spanByKey.first

(String, Seq[Int]) = (lb.b,ArrayBuffer(1, 3, 1, 1, 1))

In [33]:
val sum_by_project2 = pageCounts
    .spanByKey
    .mapValues(array => array.reduce(_+_))
    
sum_by_project2.first

(String, Int) = (lb.b,7)

In [26]:
sum_by_project2.saveToCassandra("databricks", "sum_by_project")